# Procesamiento del Lenguaje Natural

Rodrigo S. Cortez Madrigal

<img src="https://pcic.posgrado.unam.mx/wp-content/uploads/Ciencia-e-Ingenieria-de-la-Computacion_color.png" alt="Logo PCIC" width="128" />  

In [31]:
import numpy as np
import nltk
import glob
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
import plotly.express as px

In [32]:

sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

corpus_path = "corpus"  # Ruta de la carpeta donde están los archivos .txt
files = glob.glob(os.path.join(corpus_path, "*.txt"))  # Buscar todos los .txt en la carpeta
textos = []
for fn in files:
    with open(fn, encoding="utf-8") as f:  # Asegurarse de usar la codificación adecuada
        textos.append(f.read())
all_text = ' '.join(textos)
print(files)

def PredictAuthors(fvs):
    '''
    Se utiliza el algoritmo de K-Medias como método de clusterización (No supervisado)
    '''
    km = KMeans(n_clusters=2, init='k-means++', n_init=20, verbose=0)
    km.fit(fvs)
    return km
def Features(textos):
    '''
    Obtener vectores de características, utilizando características estilométricas de caracteres, léxicas y sintácticas.
    '''
    num_textos = len(textos)
    fvs = np.zeros((len(textos), 9), np.float64)
    for e, ch_text in enumerate(textos):
        contMayus = 0
        contMinus = 0
        contNumeros = 0
        textoNormal = ch_text
        textoMinusculas = ch_text.lower()
        charTotales = len(textoNormal)
        texto = ch_text.lower()
        tokens = nltk.word_tokenize(texto)
        words = word_tokenizer.tokenize(texto)
        sentences = sentence_tokenizer.tokenize(ch_text)
        vocab = set(words)
        words_per_sentence = np.array([len(word_tokenizer.tokenize(s))
                                       for s in sentences])
        for letra in textoNormal:
            if letra.isupper():
                contMayus = contMayus + 1
            else:
                contMinus = contMinus + 1
            if letra in ["1","2","3","4","5","6","7","8","9","0"]:
                contNumeros = contNumeros + 1
        # Número promedio de palabras por oración
        fvs[e, 0] = words_per_sentence.mean()
        # Variación del tamaño de las oraciones
        fvs[e, 1] = words_per_sentence.std()
        # Diversidad léxica
        fvs[e, 2] = len(vocab) / float(len(words))
        # Número de comas por oración
        fvs[e, 3] = tokens.count(',') / float(len(sentences))
        # Número de puntos y comas por oración
        fvs[e, 4] = tokens.count(';') / float(len(sentences))
        # Número de dos puntos por oración
        fvs[e, 5] = tokens.count(':') / float(len(sentences))
        # Proporción mayúsculas/total de caracteres
        fvs[e, 6] = float(contMayus/charTotales)
		# Proporción minúsculas/total de caracteres
        fvs[e, 7] = float(contMinus/charTotales)
        # Proporción de números respecto a letras
        fvs[e, 8] = float(contNumeros/charTotales)

    fvs = whiten(fvs)
    return fvs


['corpus/casatomada.txt', 'corpus/elevangeliosegunmarcos.txt', 'corpus/elaleph.txt', 'corpus/cartaaunasenoritaenparis.txt']


In [35]:
classifications = []
feature_sets = []
for element in list(Features(textos)):
    feature_sets.append(element)
feature_sets = np.array(feature_sets)
model = PredictAuthors(feature_sets)
result = model.labels_

In [36]:
print(result)

[0 0 1 0]


# Otra forma

In [90]:
import spacy

def get_Style(textos):
    lem_textos = []
    nlp = spacy.load("es_core_news_sm")
    for ch_text in textos:
        doc = nlp(ch_text)
        lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        lem_textos.append(' '.join(lemmas))

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(lem_textos)
    return X.toarray()


In [91]:
classifications = []
feature_sets = []
for element in list(get_Style(textos)):
    feature_sets.append(element)
model = PredictAuthors(feature_sets)
result = model.labels_

In [59]:
print(result)

[0 0 1 0]
